**Исходные данные:**

Таблица CsvTransactions содержит данные о безналичных платежах и проездных билетах. Для всех трех отчетов нам потребуются следующие атрибуты: 

- DesmanOperationId - уникальный код транзакции,
- DocumentCreateDate - дата,
- RouteCode - маршрут,
- VehicleNumber - номер транспортного средства,
- CardType - вид оплаты,
- TripNumber - номер рейса

Таблица Events содержит данные о продажах билетов за наличный расчет. Для всех трех отчетов нам потребуются следующие атрибуты: 

- Id - уникальный код транзакции,
- Datetime - дата,
- VehicleNumber - номер транспортного средства

Для формирования смежного отчета по двум таблицам, у таблицы Events не хватает атрибутов маршрута и номера рейса. Так как номера ТС для обеих таблиц повторяются, я предположила, что конкретный ТС ездит по уникальным рейсам и маршрутам, и их можно восстановить для таблицы Events из данных таблицы CsvTransactions. Гипотеза не подтвердилась, номера ТС не уникальны для маршрутов и рейсов. Для этих атрибутов у транзакций за наличный расчет будет применяться значение *'Нет данных'*. 

Вид оплаты для таблицы Events по умолчанию *'Наличная оплата'*.
В таблице Events присутствуют строки с предположительно некорректными значениями: VehicleNumber - номер транспортного средства со значениями -1 и 0. Только у этих записей в атрибуте EventId значения 555 и 304, предполагаю, что это ошибка транзакции на кассовом аппарате, исключим их из отчетов. 

Из таблиц справочников мы будем использовать CardType с расшифровкой кода типов оплаты. Данный атрибут фигурирует в таблице CsvTransactions. При формировании отчетов исключим код 355 - оплата наличными, так как по условиям задачи эта таблица не должна содержать оплату наличными. В предоставленном семпле данных таких строк нет, но учтем это на случай ошибки в будущем. Так же код 55 - парковка не подразумевает вид оплаты проезда на общественном транспорте. Коды 71, 350, 353, 354 отнесем к безналичной оплате разовых поездок, остальные коды - оплата проездных билетов.

**Пайплайн** будет следующий: Первый этап - выгружаем таблицы из баз с помощью пакета psycopg2. 
Второй этап - обработка и объединение таблиц. Третий этап - формированием плоской таблицы отчета. 

**В папке *test_omega*: **

- Файлы config.py и config_to_bd_example.yml (как пример конфигурации, боевой файл должен называться *config_to_bd.yml*) - для подключения к БД,
- Файл adding_tables_psycopg.py для выгрузки данных из БД,
- Файл prepare_data.py для подготовки данных и формирования отчетов.

In [6]:
import pandas as pd
from test_omega.adding_tables_psycopg import AddingDataPsycopg
from test_omega.prepare_data import *

"""Получаем новые записи из таблиц с указанием нужных дат"""
db = AddingDataPsycopg()
сsv_transactions = db.get_table_as_сsv_transactions('2020-03-14', '2020-03-15')
events = db.get_table_as_events('2020-03-14', '2020-03-15')

"""Обрабатываем и получаем сводный датасет по двум таблицам"""
df = concat_df(сsv_transactions, events)

"""Формируем нужный отчет"""
report_1(df)

,Дата,Маршрут,ТС,Вид оплаты,Количество транзакций
0,2020-03-14,10,1,Банковская карта,108
1,2020-03-14,10,1,Проездной билет,1966
2,2020-03-14,10,15,Банковская карта,107
3,2020-03-14,10,15,Проездной билет,1977
4,2020-03-14,10,21,Банковская карта,103
...,...,...,...,...,...
90,2020-03-15,Нет данных,19,Наличная оплата,113
91,2020-03-15,Нет данных,20,Наличная оплата,29
92,2020-03-15,Нет данных,21,Наличная оплата,238
93,2020-03-15,Нет данных,22,Наличная оплата,226


In [7]:
report_2(df)

,Дата,Маршрут,ТС,Номер рейса,Количество транзакций
0,2020-03-14,10,1,1,2
1,2020-03-14,10,1,2,2
2,2020-03-14,10,1,3,23
3,2020-03-14,10,1,4,6
4,2020-03-14,10,1,5,63
...,...,...,...,...,...
726,2020-03-15,Нет данных,19,Нет данных,113
727,2020-03-15,Нет данных,20,Нет данных,29
728,2020-03-15,Нет данных,21,Нет данных,238
729,2020-03-15,Нет данных,22,Нет данных,226


In [8]:
report_3(df)

,Дата,Маршрут,Наличная оплата,Банковская карта,Проездной билет
0,2020-03-14,10,0,318,5978
1,2020-03-14,155,0,652,11927
2,2020-03-14,322,0,285,6215
3,2020-03-14,323,0,945,14814
4,2020-03-14,Нет данных,96490,0,0
5,2020-03-15,10,0,3,43
6,2020-03-15,155,0,3,74
7,2020-03-15,322,0,3,47
8,2020-03-15,323,0,0,55
9,2020-03-15,Нет данных,3096,0,0
